### Proyecto Análisis de Datos
## Mushroom Dataset - Limpieza de Datos

---

## Objetivo del Proyecto

**Recordatorio:** El objetivo de este proyecto es desarrollar un **modelo predictivo** que clasifique hongos como:
- **Comestibles (e)**: Seguros para consumo humano
- **Venenosos (p)**: Peligrosos, pueden causar intoxicación

Para lograr este objetivo, necesitamos un dataset **limpio y confiable**. Este notebook documenta el proceso completo de limpieza de datos basado en los problemas identificados en el **análisis exploratorio** (AnalisisExploratorio.ipynb).

## 1. Importar Librerías y Cargar Datos

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

# Configuración
%matplotlib inline
pd.set_option('display.max_columns', None)

print("Librerías importadas")

Librerías importadas


In [36]:
# Cargar dataset original 
df_original = pd.read_csv('MushroomDataset/MushroomDataset.csv', low_memory=False, sep=',')

# Crear copia para trabajar
df = df_original.copy()

print(f"Dataset cargado: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"\nPrimeras 3 filas:")
df.head(3)

Dataset cargado: 61,079 filas × 21 columnas

Primeras 3 filas:


,class,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,p,15.26,x,NaN,o,f,e,NaN,w,16.95,17.09,s,y,w,u,w,t,g,NaN,d,w
1,p,16.6,x,g,o,f,e,NaN,w,17.99,18.19,s,y,w,u,w,t,g,NaN,d,u
2,p,14.07,NaN,NaN,o,f,e,NaN,w,17.80,17.74,s,NaN,w,NaN,w,t,g,NaN,d,w


## 2. Problemas Detectados en el Análisis Exploratorio

Según el análisis realizado en `AnalisisExploratorio.ipynb`, identificamos los siguientes problemas críticos:

### Resumen de Problemas

| Variable | Problema | Magnitud | Impacto |
|----------|----------|----------|----------|
| **cap-diameter** | Valores 'invalid_value' | 611 filas (1.00%) | Medio |
| **cap-surface** | Código 'd' no documentado | 4,234 filas (6.93%) | Alto |
| **stem-root** | Código 'f' no documentado | 1,013 filas (1.66%) | Medio |
| **veil-type** | 95.07% valores nulos | 58,066 filas | Crítico |
| **spore-print-color** | 90.13% valores nulos | 55,050 filas | Crítico |
| **veil-color** | 88.48% valores nulos | 54,045 filas | Crítico |
| **stem-root** | 85.21% valores nulos | 52,044 filas | Crítico |
| **stem-surface** | 64.40% valores nulos | 39,333 filas | Alto |
| **Duplicados** | Filas duplicadas | 45 filas (0.07%) | Bajo |
| **Outliers** | Valores extremos en numéricas | Variable | Alto |

### Objetivo de la Limpieza

Transformar este dataset problemático en uno **100% completo, sin errores, y listo para modelado predictivo**.

## 3. Estrategia de Limpieza

### Orden de Operaciones

```
PASO 1: Eliminar duplicados iniciales
   ↓
PASO 2: Eliminar variables con >85% nulos
   ↓
PASO 3: Manejar 'invalid_value' en cap-diameter
   ↓
PASO 4: Eliminar filas con valores categóricos inesperados
   ↓
PASO 5: Eliminar outliers extremos
   ↓
PASO 5.5: Imputar 'class' 
   ↓
PASO 6: Imputar valores nulos restantes (por clase)
   ↓
PASO 7: Eliminar filas muy incompletas (>10 nulos)
   ↓
PASO 7.5: Eliminar duplicados finales
   ↓
PASO 8: Resetear índice y verificación final
```

In [37]:
# Función auxiliar para reportar estado
def reportar_estado(df, titulo="Estado del Dataset"):
    print("="*70)
    print(f" {titulo}".center(70))
    print("="*70)
    print(f"Filas: {df.shape[0]:,}")
    print(f"Columnas: {df.shape[1]}")
    
    nulos_total = df.isnull().sum().sum()
    total_valores = df.shape[0] * df.shape[1]
    pct_completo = 100 - (nulos_total / total_valores * 100)
    
    print(f"Valores nulos: {nulos_total:,} ({100-pct_completo:.2f}%)")
    print(f"Completitud: {pct_completo:.2f}%")
    
    if 'class' in df.columns:
        dist = df['class'].value_counts()
        print(f"\nBalance de clases:")
        for clase, count in dist.items():
            print(f"  {clase}: {count:,} ({count/len(df)*100:.2f}%)")
    print("="*70)

# Estado inicial
reportar_estado(df, "ESTADO INICIAL")

                            ESTADO INICIAL                            
Filas: 61,079
Columnas: 21
Valores nulos: 356,255 (27.77%)
Completitud: 72.23%

Balance de clases:
  p: 32,215 (52.74%)
  e: 25,811 (42.26%)


---

##  INICIO DEL PROCESO DE LIMPIEZA

---

## PASO 1: Eliminar Duplicados Iniciales

- Solo son **45 filas (0.07%)** - impacto mínimo en cantidad de datos

### Decisión: ELIMINAR

Después de eliminar columnas, pueden aparecer nuevos duplicados. Los eliminaremos de nuevo en el PASO 7.5 (al final).

In [39]:
# Identificar duplicados
duplicados = df.duplicated()
num_duplicados = duplicados.sum()

print(f"Duplicados encontrados: {num_duplicados} ({num_duplicados/len(df)*100:.2f}%)")

if num_duplicados > 0:
    # Eliminar
    filas_antes = len(df)
    df = df.drop_duplicates(keep='first').copy()
    print(f" Eliminados: {num_duplicados}")
    print(f" Restantes: {len(df):,}")
else:
    print(" No hay duplicados")

Duplicados encontrados: 0 (0.00%)
 No hay duplicados


## PASO 2: Eliminar Variables con >85% Valores Nulos

### ¿Por qué 85% como umbral?

Cuando una variable tiene más del 85% de valores faltantes:
- **Solo 15% o menos** son datos reales
- **Imputar 85%** significa crear datos sintéticos que NO existen
- Puede introducir **patrones falsos** en el modelo
- El modelo aprende de datos inventados, no reales

### Variables a eliminar (según AnalisisExploratorio.ipynb):

1. **veil-type**: 95.07% nulos
2. **spore-print-color**: 90.13% nulos
3. **veil-color**: 88.48% nulos
4. **stem-root**: 85.21% nulos

### Decisión: ELIMINAR estas 4 variables

In [40]:
# Identificar variables con >85% nulos
umbral = 0.85
nulos_pct = df.isnull().sum() / len(df)
vars_eliminar = nulos_pct[nulos_pct > umbral].index.tolist()

print(f"Variables con >{umbral*100}% de valores nulos:")
print("="*70)
for var in vars_eliminar:
    pct = nulos_pct[var] * 100
    nulos = df[var].isnull().sum()
    print(f"  {var:25s}: {nulos:6,} nulos ({pct:5.2f}%)")

print(f"\nSe eliminarán {len(vars_eliminar)} variables")

Variables con >85.0% de valores nulos:
  stem-root                : 52,032 nulos (85.25%)
  veil-type                : 58,021 nulos (95.06%)
  veil-color               : 54,002 nulos (88.48%)
  spore-print-color        : 55,018 nulos (90.14%)

Se eliminarán 4 variables


In [41]:
# Eliminar
df = df.drop(columns=vars_eliminar)

print(f"Variables eliminadas: {len(vars_eliminar)}")
print(f"Variables restantes: {df.shape[1]}")
print(f"Nuevas dimensiones: {df.shape[0]:,} × {df.shape[1]}")

Variables eliminadas: 4
Variables restantes: 17
Nuevas dimensiones: 61,034 × 17


## PASO 3: Manejar 'invalid_value' en cap-diameter

### Problema Detectado

- **611 filas (1.00%)** tienen el string `'invalid_value'` en lugar de un número
- La variable no es numérica (dtype: object)

### Decisión: IMPUTAR con mediana por clase (e/p)

Los hongos comestibles y venenosos tienen tamaños diferentes. Imputar por clase preserva estas diferencias morfológicas.

In [43]:
# Analizar problema
filas_invalidas = df['cap-diameter'] == 'invalid_value'
num_invalidos = filas_invalidas.sum()

print(f"Filas con 'invalid_value': {num_invalidos} ({num_invalidos/len(df)*100:.2f}%)")
print(f"\nDistribución por clase:")
print(df[filas_invalidas]['class'].value_counts())

Filas con 'invalid_value': 611 (1.00%)

Distribución por clase:
class
p    289
e    283
Name: count, dtype: int64


In [44]:
# Convertir a numérico (invalid_value → NaN)
df['cap-diameter'] = pd.to_numeric(df['cap-diameter'], errors='coerce')

print(f"cap-diameter convertido a numérico")
print(f"Tipo: {df['cap-diameter'].dtype}")
print(f"NaN generados: {df['cap-diameter'].isnull().sum()}")

cap-diameter convertido a numérico
Tipo: float64
NaN generados: 3611


In [45]:
# Calcular medianas por clase (excluyendo outliers extremos)
medianas_clase = df[df['cap-diameter'] < 100].groupby('class')['cap-diameter'].median()

print("Medianas por clase:")
for clase, mediana in medianas_clase.items():
    print(f"  Clase {clase}: {mediana:.2f} cm")

# Imputar
for clase in df['class'].dropna().unique():
    mascara = (df['class'] == clase) & (df['cap-diameter'].isnull())
    num_imputados = mascara.sum()
    if num_imputados > 0:
        df.loc[mascara, 'cap-diameter'] = medianas_clase[clase]
        print(f"\nClase '{clase}': {num_imputados} valores imputados con {medianas_clase[clase]:.2f}")

print(f"\nNulos restantes en cap-diameter: {df['cap-diameter'].isnull().sum()}")

Medianas por clase:
  Clase e: 6.71 cm
  Clase p: 4.98 cm

Clase 'p': 1848 valores imputados con 4.98

Clase 'e': 1586 valores imputados con 6.71

Nulos restantes en cap-diameter: 177


## PASO 4: Eliminar Filas con Valores Categóricos Inesperados

### Problemas Detectados (AnalisisExploratorio.ipynb)

1. **cap-surface**: código `'d'` NO está en la metadata (4,234 filas, 6.93%)
2. **stem-root**: código `'f'` NO está en la metadata (1,013 filas, 1.66%)

### Decisión: ELIMINAR estas filas

**Razón:** Para un modelo de **salud pública** (predecir hongos venenosos), la precisión es crítica. Datos dudosos pueden causar predicciones incorrectas peligrosas.

In [48]:
# Definir valores esperados (de la metadata)
valores_esperados = {
    'cap-surface': ['i', 'g', 'y', 's', 'h', 'l', 'k', 't', 'w', 'e'],
    'stem-surface': ['i', 'g', 'y', 's', 'h', 'l', 'k', 't', 'w', 'e', 'f'],
}

# Identificar filas problemáticas
filas_eliminar = pd.Series([False] * len(df), index=df.index)

print("Identificando valores inesperados...")
print("="*70)

# cap-surface
if 'cap-surface' in df.columns:
    mascara = df['cap-surface'].notna() & ~df['cap-surface'].isin(valores_esperados['cap-surface'])
    num = mascara.sum()
    if num > 0:
        valores = df.loc[mascara, 'cap-surface'].unique()
        print(f"\ncap-surface:")
        print(f"  Valores inesperados: {list(valores)}")
        print(f"  Filas afectadas: {num:,} ({num/len(df)*100:.2f}%)")
        filas_eliminar = filas_eliminar | mascara

# stem-surface
if 'stem-surface' in df.columns:
    mascara = df['stem-surface'].notna() & ~df['stem-surface'].isin(valores_esperados['stem-surface'])
    num = mascara.sum()
    if num > 0:
        valores = df.loc[mascara, 'stem-surface'].unique()
        print(f"\nstem-surface:")
        print(f"  Valores inesperados: {list(valores)}")
        print(f"  Filas afectadas: {num:,} ({num/len(df)*100:.2f}%)")
        filas_eliminar = filas_eliminar | mascara

print(f"\nTotal a eliminar: {filas_eliminar.sum():,} ({filas_eliminar.sum()/len(df)*100:.2f}%)")

Identificando valores inesperados...

Total a eliminar: 0 (0.00%)


In [49]:
# Eliminar filas
filas_antes = len(df)
df = df[~filas_eliminar].copy()

print(f"- Filas eliminadas: {filas_antes - len(df):,}")
print(f"- Filas restantes: {len(df):,}")

- Filas eliminadas: 0
- Filas restantes: 56,801


## PASO 5: Eliminar Outliers Extremos

### Problema Detectado (AnalisisExploratorio.ipynb)

- **cap-diameter max**: 623.40 cm 
- **stem-width max**: 1,039.10 mm  

Estos valores son **biológicamente imposibles**.

### Método: IQR × 3 (conservador)

```
IQR = Q3 - Q1
Límite superior = Q3 + 3 × IQR
```

Factor **3** es más conservador que el estándar 1.5 - solo elimina valores **extremadamente** atípicos.

### Decisión: ELIMINAR outliers

In [50]:
def detectar_outliers_iqr(serie, factor=3):
    """Detecta outliers usando método IQR. Retorna máscara con índices del DataFrame completo."""
    # Calcular estadísticas solo sobre valores no nulos
    valores_no_nulos = serie.dropna()
    Q1 = valores_no_nulos.quantile(0.25)
    Q3 = valores_no_nulos.quantile(0.75)
    IQR = Q3 - Q1
    lim_inf = Q1 - factor * IQR
    lim_sup = Q3 + factor * IQR
    
    # Crear máscara sobre la serie completa (mantiene índices originales)
    outliers = (serie < lim_inf) | (serie > lim_sup)
    
    return outliers, lim_inf, lim_sup

vars_numericas = ['cap-diameter', 'stem-height', 'stem-width']
filas_outliers = pd.Series([False] * len(df), index=df.index)

print("Detectando outliers extremos (IQR × 3)...")
print("="*70)

for var in vars_numericas:
    if var in df.columns:
        # Detectar outliers
        outliers, lim_inf, lim_sup = detectar_outliers_iqr(df[var], factor=3)
        num = outliers.sum()
        
        print(f"\n{var}:")
        print(f"  Límite superior: {lim_sup:.2f}")
        print(f"  Outliers: {num} ({num/len(df)*100:.3f}%)")
        
        if num > 0:
            # Ahora los índices coinciden
            max_outlier = df.loc[outliers, var].max()
            print(f"  Max outlier: {max_outlier:.2f}")
            filas_outliers = filas_outliers | outliers

print(f"\n Total filas con outliers: {filas_outliers.sum():,}")

Detectando outliers extremos (IQR × 3)...

cap-diameter:
  Límite superior: 23.24
  Outliers: 971 (1.709%)
  Max outlier: 623.40

stem-height:
  Límite superior: 17.53
  Outliers: 1438 (2.532%)
  Max outlier: 339.20

stem-width:
  Límite superior: 52.15
  Outliers: 1043 (1.836%)
  Max outlier: 1039.10

 Total filas con outliers: 3,222


In [51]:
# Eliminar outliers
filas_antes = len(df)
df = df[~filas_outliers].copy()

print(f" Filas eliminadas: {filas_antes - len(df):,}")
print(f" Filas restantes: {len(df):,}")

# Mostrar nuevos rangos
print(f"\nNuevos rangos:")
for var in vars_numericas:
    if var in df.columns:
        print(f"  {var:15s}: {df[var].min():.2f} - {df[var].max():.2f}")

 Filas eliminadas: 3,222
 Filas restantes: 53,579

Nuevos rangos:
  cap-diameter   : 0.38 - 23.16
  stem-height    : 0.00 - 17.53
  stem-width     : 0.00 - 51.93


## PASO 5.5: Imputar 'class' ANTES de las demás variables

**¿Por qué?** La imputación del PASO 6 usa `class` para agrupar. Si `class` tiene nulos, esas filas no se imputan y quedan con valores faltantes.

In [52]:
# Imputar 'class' con la moda ANTES de imputar otras variables
if df['class'].isnull().sum() > 0:
    num_nulos = df['class'].isnull().sum()
    moda_class = df['class'].mode()[0]
    df['class'].fillna(moda_class, inplace=True)
    print(f" Variable 'class' imputada: {num_nulos:,} → '{moda_class}'")
    print(f" Nulos restantes en 'class': {df['class'].isnull().sum()}")
else:
    print(" Variable 'class' no tiene nulos")

 Variable 'class' imputada: 2,695 → 'p'
 Nulos restantes en 'class': 0


In [53]:
# Imputar variables numéricas con MEDIANA por clase
print("Imputando numéricas (mediana por clase)...")
print("="*70)

for var in vars_numericas:
    if var in df.columns and df[var].isnull().sum() > 0:
        for clase in df['class'].unique():  # Ya no hay nulos en class
            mascara = (df['class'] == clase) & (df[var].isnull())
            num = mascara.sum()
            if num > 0:
                # Calcular mediana SOLO sobre valores no nulos
                valores_validos = df[(df['class'] == clase) & (df[var].notna())][var]
                
                if len(valores_validos) > 0:
                    mediana = valores_validos.median()
                    df.loc[mascara, var] = mediana
                    print(f"  {var} ({clase}): {num:,} imputados → {mediana:.2f}")
                else:
                    # Si no hay valores válidos para esta clase, usar mediana global
                    mediana_global = df[var].median()
                    df.loc[mascara, var] = mediana_global
                    print(f"  {var} ({clase}): {num:,} imputados → {mediana_global:.2f} (mediana global)")

Imputando numéricas (mediana por clase)...
  cap-diameter (p): 161 imputados → 4.98
  stem-height (p): 1,577 imputados → 5.70
  stem-height (e): 1,143 imputados → 6.16
  stem-width (p): 1,548 imputados → 7.85
  stem-width (e): 1,145 imputados → 12.11


In [54]:
# Verificar que no queden nulos en numéricas
print("\n" + "="*70)
print("Verificación de variables numéricas:")
for var in vars_numericas:
    if var in df.columns:
        nulos = df[var].isnull().sum()
        print(f"  {var}: {nulos} nulos")
        
if df[vars_numericas].isnull().sum().sum() == 0:
    print("\n Todas las variables numéricas imputadas correctamente")
else:
    print(f"\n  Aún quedan {df[vars_numericas].isnull().sum().sum()} nulos en variables numéricas")


Verificación de variables numéricas:
  cap-diameter: 0 nulos
  stem-height: 0 nulos
  stem-width: 0 nulos

 Todas las variables numéricas imputadas correctamente


In [55]:
# Imputar variables categóricas con MODA por clase
print("\nImputando categóricas (moda por clase)...")
print("="*70)

vars_cat = df.select_dtypes(include=['object']).columns.tolist()
if 'class' in vars_cat:
    vars_cat.remove('class')

for var in vars_cat:
    if df[var].isnull().sum() > 0:
        for clase in df['class'].unique():  # Ya no hay nulos en class
            mascara = (df['class'] == clase) & (df[var].isnull())
            num = mascara.sum()
            if num > 0:
                # Obtener valores válidos (no nulos) para esta clase
                valores_validos = df[(df['class'] == clase) & (df[var].notna())][var]
                
                if len(valores_validos) > 0:
                    moda = valores_validos.mode()[0] if len(valores_validos.mode()) > 0 else valores_validos.iloc[0]
                    df.loc[mascara, var] = moda
                    print(f"  {var} ({clase}): {num:,} imputados → '{moda}'")
                else:
                    # Si no hay valores válidos para esta clase, usar moda global
                    moda_global = df[var].mode()[0] if len(df[var].mode()) > 0 else 'unknown'
                    df.loc[mascara, var] = moda_global
                    print(f"  {var} ({clase}): {num:,} imputados → '{moda_global}' (moda global)")

# Verificar que no queden nulos en categóricas
print("\n" + "="*70)
print("Verificación de variables categóricas:")
nulos_cat = df[vars_cat].isnull().sum()
nulos_cat = nulos_cat[nulos_cat > 0]

if len(nulos_cat) == 0:
    print("Todas las variables categóricas imputadas correctamente")
else:
    print("Variables categóricas con nulos:")
    for var, count in nulos_cat.items():
        print(f"  {var}: {count}")


Imputando categóricas (moda por clase)...
  cap-shape (p): 1,629 imputados → 'x'
  cap-shape (e): 1,069 imputados → 'x'
  cap-surface (p): 8,811 imputados → 't'
  cap-surface (e): 7,068 imputados → 's'
  cap-color (p): 1,570 imputados → 'n'
  cap-color (e): 1,120 imputados → 'n'
  does-bruise-or-bleed (p): 1,582 imputados → 'f'
  does-bruise-or-bleed (e): 1,102 imputados → 'f'
  gill-attachment (p): 7,568 imputados → 'a'
  gill-attachment (e): 3,843 imputados → 'a'
  gill-spacing (p): 14,535 imputados → 'c'
  gill-spacing (e): 8,747 imputados → 'c'
  gill-color (p): 1,552 imputados → 'w'
  gill-color (e): 1,121 imputados → 'w'
  stem-surface (p): 19,036 imputados → 'y'
  stem-surface (e): 15,420 imputados → 's'
  stem-color (p): 1,584 imputados → 'n'
  stem-color (e): 1,095 imputados → 'w'
  has-ring (p): 1,534 imputados → 'f'
  has-ring (e): 1,146 imputados → 'f'
  ring-type (p): 2,532 imputados → 'f'
  ring-type (e): 2,263 imputados → 'f'
  habitat (p): 1,597 imputados → 'd'
  habit

In [56]:
# Verificar que la imputación fue exitosa
nulos_final = df.isnull().sum().sum()
print(f"{'='*70}")
print(f"Valores nulos después de imputación: {nulos_final}")

if nulos_final == 0:
    print("Dataset 100% completo")
else:
    print(f"Aún quedan {nulos_final} nulos")
    # Mostrar cuáles variables tienen nulos
    nulos_restantes = df.isnull().sum()
    nulos_restantes = nulos_restantes[nulos_restantes > 0]
    if len(nulos_restantes) > 0:
        print(f"\nVariables con nulos:")
        for var, count in nulos_restantes.items():
            print(f"  {var}: {count:,}")

Valores nulos después de imputación: 0
Dataset 100% completo


In [57]:
# Identificar filas con cualquier nulo restante
nulos_por_fila = df.isnull().sum(axis=1)
filas_con_nulos = nulos_por_fila > 0
num_filas_con_nulos = filas_con_nulos.sum()

print(f"Filas con al menos 1 nulo: {num_filas_con_nulos}")

if num_filas_con_nulos > 0:
    # Mostrar distribución de nulos
    print(f"\nDistribución de nulos por fila:")
    dist_nulos = nulos_por_fila[nulos_por_fila > 0].value_counts().sort_index()
    for num_nulos, count in dist_nulos.items():
        print(f"  {num_nulos} nulos: {count} filas")
    
    # Eliminar todas las filas con nulos
    filas_antes = len(df)
    df = df[~filas_con_nulos].copy()
    print(f"\n Filas eliminadas: {filas_antes - len(df):,}")
    print(f" Filas restantes: {len(df):,}")
    
    # Verificar
    nulos_restantes = df.isnull().sum().sum()
    if nulos_restantes == 0:
        print("\n Dataset ahora está 100% completo (sin nulos)")
    else:
        print(f"\n  ERROR: Aún quedan {nulos_restantes} nulos")
else:
    print(" No hay filas con nulos - dataset 100% completo")

Filas con al menos 1 nulo: 0
 No hay filas con nulos - dataset 100% completo


## PASO 7.5: Eliminar Duplicados (AL FINAL)

**¿Por qué al final?**

Cuando eliminamos columnas (PASO 2), filas que antes eran diferentes se pueden volver idénticas. Por eso necesitamos eliminar duplicados **DESPUÉS** de todas las transformaciones.

In [58]:
# Verificar y eliminar duplicados finales
duplicados_finales = df.duplicated()
num_duplicados = duplicados_finales.sum()

print(f"Duplicados encontrados después de todas las transformaciones: {num_duplicados}")

if num_duplicados > 0:
    filas_antes = len(df)
    df = df.drop_duplicates(keep='first').copy()
    print(f" Duplicados eliminados: {num_duplicados}")
    print(f" Filas restantes: {len(df):,}")
else:
    print(" No hay duplicados")

Duplicados encontrados después de todas las transformaciones: 38
 Duplicados eliminados: 38
 Filas restantes: 53,541


## PASO 7: Eliminar Filas Muy Incompletas

Si después de imputar quedan filas con muchos nulos (>10), las eliminamos.

**Nota:** Este paso probablemente no hará nada si la imputación fue exitosa.

In [59]:
nulos_por_fila = df.isnull().sum(axis=1)
filas_problematicas = nulos_por_fila > 10
num = filas_problematicas.sum()

print(f"Filas con >10 nulos: {num}")

if num > 0:
    df = df[~filas_problematicas].copy()
    print(f" Eliminadas: {num}")
else:
    print(" No hay filas muy incompletas")

Filas con >10 nulos: 0
 No hay filas muy incompletas


## PASO 8: Resetear Índice y Finalizar

In [60]:
# Resetear índice
df = df.reset_index(drop=True)

print(f" Índice reseteado: 0 a {len(df)-1}")

 Índice reseteado: 0 a 53540


---

##  LIMPIEZA COMPLETADA

---

## 4. Validación del Dataset Limpio

In [61]:
reportar_estado(df, "ESTADO FINAL")

                             ESTADO FINAL                             
Filas: 53,541
Columnas: 17
Valores nulos: 0 (0.00%)
Completitud: 100.00%

Balance de clases:
  p: 31,539 (58.91%)
  e: 22,002 (41.09%)


In [62]:
# Checklist de validación
print("\n" + "="*70)
print(" CHECKLIST DE VALIDACIÓN".center(70))
print("="*70)

checks = []

# 1. cap-diameter es numérico
check1 = df['cap-diameter'].dtype in ['float64', 'int64']
checks.append(check1)
print(f"\n1. {'' if check1 else ''} cap-diameter es numérico ({df['cap-diameter'].dtype})")

# 2. Sin nulos
check2 = df.isnull().sum().sum() == 0
checks.append(check2)
print(f"2. {'' if check2 else ''} Sin valores nulos")

# 3. Sin duplicados
check3 = df.duplicated().sum() == 0
checks.append(check3)
print(f"3. {'' if check3 else ''} Sin duplicados")

# 4. Variables críticas eliminadas
vars_criticas = ['veil-type', 'spore-print-color', 'veil-color', 'stem-root']
check4 = all(v not in df.columns for v in vars_criticas)
checks.append(check4)
print(f"4. {'' if check4 else ''} Variables con >85% nulos eliminadas")

# 5. Sin outliers extremos
check5 = df['cap-diameter'].max() < 100 and df['stem-width'].max() < 200
checks.append(check5)
print(f"5. {'' if check5 else ''} Sin outliers extremos")

# 6. Índice correcto
check6 = df.index.tolist() == list(range(len(df)))
checks.append(check6)
print(f"6. {'' if check6 else ''} Índice reseteado")

# 7. Conserva >50% de datos
check7 = len(df) > (len(df_original) * 0.5)
checks.append(check7)
pct = (len(df) / len(df_original)) * 100
print(f"7. {'' if check7 else ''} Conserva {pct:.2f}% de datos originales")

# 8. Balance razonable
if 'class' in df.columns:
    dist = df['class'].value_counts(normalize=True)
    check8 = all((dist >= 0.3) & (dist <= 0.7))
    checks.append(check8)
    print(f"8. {'' if check8 else ''} Balance de clases razonable")

# Resultado
print("\n" + "="*70)


                        CHECKLIST DE VALIDACIÓN                       

1.  cap-diameter es numérico (float64)
2.  Sin valores nulos
3.  Sin duplicados
4.  Variables con >85% nulos eliminadas
5.  Sin outliers extremos
6.  Índice reseteado
7.  Conserva 87.66% de datos originales
8.  Balance de clases razonable



In [63]:
print("\n" + "="*70)
print(" COMPARACIÓN: ORIGINAL vs LIMPIO".center(70))
print("="*70)

comparacion = pd.DataFrame({
    'Métrica': [
        'Filas',
        'Columnas',
        'Valores nulos',
        '% Completitud',
        'Duplicados'
    ],
    'Original': [
        f"{len(df_original):,}",
        f"{df_original.shape[1]}",
        f"{df_original.isnull().sum().sum():,}",
        f"{100 - (df_original.isnull().sum().sum() / (df_original.shape[0] * df_original.shape[1]) * 100):.2f}%",
        f"{df_original.duplicated().sum()}"
    ],
    'Limpio': [
        f"{len(df):,}",
        f"{df.shape[1]}",
        f"{df.isnull().sum().sum():,}",
        f"{100 - (df.isnull().sum().sum() / (df.shape[0] * df.shape[1]) * 100):.2f}%",
        f"{df.duplicated().sum()}"
    ],
    'Cambio': [
        f"{len(df) - len(df_original):,} ({((len(df) - len(df_original)) / len(df_original) * 100):.2f}%)",
        f"{df.shape[1] - df_original.shape[1]}",
        f"{df.isnull().sum().sum() - df_original.isnull().sum().sum():,}",
        f"+{100 - (df_original.isnull().sum().sum() / (df_original.shape[0] * df_original.shape[1]) * 100):.2f}pp",
        f"{df.duplicated().sum() - df_original.duplicated().sum()}"
    ]
})

print("\n")
print(comparacion.to_string(index=False))
print("\n" + "="*70)


                    COMPARACIÓN: ORIGINAL vs LIMPIO                   


      Métrica Original  Limpio           Cambio
        Filas   61,079  53,541 -7,538 (-12.34%)
     Columnas       21      17               -4
Valores nulos  356,255       0         -356,255
% Completitud   72.23% 100.00%         +72.23pp
   Duplicados       45       0              -45



## 6. Guardar Dataset Limpio

In [64]:
# Guardar dataset limpio
output_path = 'MushroomDataset/MushroomDataset_cleaned.csv'
df.to_csv(output_path, index=False, sep=';')

print(f" Dataset guardado en: {output_path}")

 Dataset guardado en: MushroomDataset/MushroomDataset_cleaned.csv


## 7. Resumen de Decisiones

### Tabla de Decisiones

| Problema | Solución | Filas Afectadas | Justificación |
|----------|----------|-----------------|---------------|
| **Duplicados** | ELIMINAR | 45 (0.07%) | Evitar data leakage |
| **Variables >85% nulos** | ELIMINAR 4 vars | - | Evitar datos sintéticos |
| **'invalid_value'** | IMPUTAR mediana | 611 (1.00%) | Solo 1%, preservar info |
| **Códigos inesperados** | ELIMINAR filas | ~4,200 (6.93%) | Datos incorrectos peores que menos datos |
| **Outliers extremos** | ELIMINAR (IQR×3) | ~100 | Biológicamente imposibles |
| **Nulos restantes** | IMPUTAR por clase | Variable | Preservar diferencias e/p |
